In [1]:
# step 3
from nilearn import datasets
from nilearn import image, plotting
from nilearn.input_data import NiftiLabelsMasker
from alive_progress import alive_bar
from tqdm import tqdm
import nibabel as nib
import pandas as pd
import numpy as np
#from hm_tools import *
import scipy.io 
import h5py
import mne
import sys
import os
from glob import glob
sys.path.insert(0, 'E:/workspace/my_py_toolbox/')
from hm_tools import *

data_path = "E:/workspace/OTface/roi_data/aal_smooth6/"
result_path = "E:/workspace/OTface/roi_data/aal_smooth6/cpm/"
condition_name = ['OT', 'PL']
file_name = ['task', 'rest']

# mapping from subject id to group
sub2group = dict()
[sub2group.update({f"s{path[-3:]}": group}) for group in ["OT", "PL"] for path in glob(os.path.join(data_path, "task", "preprocess", group, "sub*"))]
print(sub2group)

behavior_df = pd.read_csv("E:/workspace/OTface/behavior_metrics.csv")


{}


In [ ]:
import vlc

In [2]:
hm_shutdown(600)

In [ ]:
# 90 aal rois
roi_name = np.array(['Precentral_L','Precentral_R',
 'Frontal_Sup_L', 'Frontal_Sup_R',
 'Frontal_Sup_Orb_L', 'Frontal_Sup_Orb_R',
 'Frontal_Mid_L', 'Frontal_Mid_R',
 'Frontal_Mid_Orb_L', 'Frontal_Mid_Orb_R',
 'Frontal_Inf_Oper_L', 'Frontal_Inf_Oper_R',
 'Frontal_Inf_Tri_L', 'Frontal_Inf_Tri_R',
 'Frontal_Inf_Orb_L', 'Frontal_Inf_Orb_R',
 'Rolandic_Oper_L', 'Rolandic_Oper_R',
 'Supp_Motor_Area_L', 'Supp_Motor_Area_R',
 'Olfactory_L', 'Olfactory_R',
 'Frontal_Sup_Medial_L', 'Frontal_Sup_Medial_R',
 'Frontal_Med_Orb_L', 'Frontal_Med_Orb_R',
 'Rectus_L', 'Rectus_R',
 'Insula_L', 'Insula_R',
 'Cingulum_Ant_L', 'Cingulum_Ant_R',
 'Cingulum_Mid_L', 'Cingulum_Mid_R',
 'Cingulum_Post_L', 'Cingulum_Post_R',
 'Hippocampus_L', 'Hippocampus_R',
 'ParaHippocampal_L', 'ParaHippocampal_R',
 'Amygdala_L', 'Amygdala_R',
 'Calcarine_L', 'Calcarine_R',
 'Cuneus_L', 'Cuneus_R',
 'Lingual_L', 'Lingual_R',
 'Occipital_Sup_L', 'Occipital_Sup_R',
 'Occipital_Mid_L', 'Occipital_Mid_R',
 'Occipital_Inf_L', 'Occipital_Inf_R',
 'Fusiform_L', 'Fusiform_R',
 'Postcentral_L', 'Postcentral_R',
 'Parietal_Sup_L', 'Parietal_Sup_R',
 'Parietal_Inf_L', 'Parietal_Inf_R',
 'SupraMarginal_L', 'SupraMarginal_R',
 'Angular_L', 'Angular_R',
 'Precuneus_L', 'Precuneus_R',
 'Paracentral_Lobule_L', 'Paracentral_Lobule_R',
 'Caudate_L', 'Caudate_R',
 'Putamen_L', 'Putamen_R',
 'Pallidum_L', 'Pallidum_R',
 'Thalamus_L', 'Thalamus_R',
 'Heschl_L', 'Heschl_R',
 'Temporal_Sup_L', 'Temporal_Sup_R',
 'Temporal_Pole_Sup_L', 'Temporal_Pole_Sup_R',
 'Temporal_Mid_L', 'Temporal_Mid_R',
 'Temporal_Pole_Mid_L', 'Temporal_Pole_Mid_R',
 'Temporal_Inf_L', 'Temporal_Inf_R',])

atlas = datasets.fetch_atlas_aal()
print('Power atlas comes with {0}.'.format(atlas.keys()))
all_location = []
for iter_roi_name in roi_name:
    location = np.where(np.array(atlas['labels']) == iter_roi_name)[0][0]
    all_location.append(location)

all_location = np.array(all_location)
print(all_location)

In [ ]:
# 得到rest和task时的功能连接，并且只留下半矩阵并且铺平
filename = ['task', 'rest']
all_data = {}
for iter_file in filename:

    iter_file_data = {}

    fmri_data = np.load(data_path + iter_file + '.npy', allow_pickle=True).item()  # subjects * time * aal rois

    for iter_condition in condition_name:

        iter_condition_coor_matrix = []
        for iter_subject in (range(len(fmri_data[iter_condition]))):
            # 数据维度为 times * 28 ROI
            iter_subjectfmri_data = np.array(fmri_data[iter_condition])[iter_subject, all_location]
            # 把数据维度转换为 ROI * times
            iter_subjectfmri_data = np.transpose(iter_subjectfmri_data)


            iter_data = iter_subjectfmri_data[all_location]
            # ROI两两相关
            iter_coor_matrix = np.corrcoef(iter_data)
            # 去除对角线
            # skip_diag_matrix = skip_diag_masking(iter_coor_matrix)
            # 选取六个ifg区域
            # iter_ifg_matrix = skip_diag_matrix[ifg_location]
            iter_coor_matrix = np.tril(iter_coor_matrix, k=-1)
            iter_coor_matrix = iter_coor_matrix.flatten()
            iter_coor_matrix = iter_coor_matrix[iter_coor_matrix!=0]

            iter_condition_coor_matrix.append(iter_coor_matrix)

        iter_condition_coor_matrix = np.array(iter_condition_coor_matrix)
        iter_file_data[iter_condition] = iter_condition_coor_matrix
    all_data[iter_file] = iter_file_data
all_data['roi_name'] = roi_name

print(np.array(all_data['task']['OT']).shape)
print(np.array(all_data['rest']['PL']).shape)
all_data['rest'] = np.vstack((all_data['rest']['OT'], all_data['rest']['PL']))
all_data['task'] = np.vstack((all_data['task']['OT'], all_data['task']['PL']))
print(np.array(all_data['rest']).shape)
# 找到每个边的命名，并且只取下半部分
all_fc_name = []
for iter_x_location in roi_name:
    iter_x = []
    for iter_y_location in roi_name:
        name = iter_x_location+ '_'+iter_y_location
        iter_x.append(name)
    all_fc_name.append(iter_x)
all_fc_name = np.array(all_fc_name)
all_fc_name = np.tril(all_fc_name, k=-1)
all_fc_name = all_fc_name.flatten()
all_fc_name = all_fc_name[all_fc_name!='']
print(all_fc_name.shape)
#储存
all_data['all_fc_name'] = all_fc_name

# hm_make_dir(result_path)
# np.save(result_path + '/' + 'aal_90roi_fc.npy', all_data)

In [ ]:
# Select significant sides (rest)  OT and PL
# create label
from tqdm import trange

threshold = 0.01
behavior_data = 'rt_std'
selected_feature = {}
selected_feature['OT'] = {}
selected_feature['PL'] = {}

# for rest
all_significant_num = []
for iter_fc in trange(all_data['rest'].shape[1]):

    # 提取输入数据
    y = np.array(behavior_df[behavior_data])[0:30]
    x = all_data['rest'][0:30, iter_fc]

    scipy_corr = scipy.stats.pearsonr(x, y)

    if scipy_corr[1] < threshold:
        all_significant_num.append(iter_fc)
    # 
selected_feature['OT']['feature'] = all_data['rest'][0:30, np.array(all_significant_num)]
selected_feature['OT']['feature_name'] = all_fc_name[np.array(all_significant_num)]

print(selected_feature['OT']['feature_name'].shape)

# Select significant sides (rest)   PL
all_significant_num = []
for iter_fc in trange(all_data['rest'].shape[1]):

    # 提取输入数据
    y = np.array(behavior_df[behavior_data])[30:]
    x = all_data['rest'][30:,iter_fc]

    scipy_corr = scipy.stats.pearsonr(x, y)

    if scipy_corr[1] < threshold:
        all_significant_num.append(iter_fc)
    # 
selected_feature['PL']['feature'] = all_data['rest'][30:, np.array(all_significant_num)]
selected_feature['PL']['feature_name'] = all_fc_name[np.array(all_significant_num)]
print(selected_feature['PL']['feature_name'].shape)


In [ ]:
#进行单次 OT svm  for OT
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
lasso = Lasso()
etn = ElasticNet()

scaler = StandardScaler()

# import data
dataset = scaler.fit_transform(selected_feature['OT']['feature']) 
# dataset = selected_feature['OT']['feature']    

y = np.array(behavior_df[behavior_data])[0:30]
############################SVM+LOOVC################
loo = LeaveOneOut()
loo.get_n_splits(dataset)

predictlabel_list = []
reallabel_list = []

# C：C-SVC的惩罚参数C
# kernel: linear, poly, rbf, sigmoid, precomputed
C = 1
clf = SVR(C=C, kernel='linear', gamma='auto', degree=6, epsilon = 0.02)
# clf = lasso
# 用留一法进行验证
for train_index, test_index in loo.split(dataset):
    # 分割训练集
    X_train, X_test = dataset[train_index], dataset[test_index]
    # 分割测试集
    Y_train, Y_test = y[np.array(train_index)], y[np.array(test_index)]
    # 拟合
    clf.fit(X_train, Y_train)
    # 储存训练结果
    predictlabel_list.append(list(clf.predict(X_test)))
    reallabel_list.append(list(Y_test))
reallabel_list = np.squeeze(reallabel_list)
predictlabel_list = np.squeeze(predictlabel_list)

scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
print('******循环结束！************')
print('准确率为：',  (scipy_corr))
print('******运行结束！************')

In [ ]:
#进行单次 OT svm for pl
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score

scaler = StandardScaler()

# import data
dataset = scaler.fit_transform(selected_feature['PL']['feature']) 
y = np.array(behavior_df[behavior_data])[30:]
# dataset = selected_feature['PL']['feature']    
############################SVM+LOOVC################
loo = LeaveOneOut()
loo.get_n_splits(dataset)
predictlabel_list_pl = []
reallabel_list_pl = []

# C：C-SVC的惩罚参数C
# kernel: linear, poly, rbf, sigmoid, precomputed
C = 1
clf = SVR(C=C, kernel='linear', gamma='auto', degree=6, epsilon = 0.02)

# 用留一法进行验证
for train_index, test_index in loo.split(dataset):
    # 分割训练集
    X_train, X_test = dataset[train_index], dataset[test_index]
    # 分割测试集
    Y_train, Y_test = y[np.array(train_index)], y[np.array(test_index)]
    # 拟合
    clf.fit(X_train, Y_train)
    # 储存训练结果
    predictlabel_list_pl.append(list(clf.predict(X_test)))
    reallabel_list_pl.append(list(Y_test))
reallabel_list_pl = np.squeeze(reallabel_list_pl)
predictlabel_list_pl = np.squeeze(predictlabel_list_pl)
# accurancy = count_right_label / len(label)
scipy_corr = scipy.stats.spearmanr(reallabel_list_pl, predictlabel_list_pl)
print('******循环结束！************')
print('准确率为：',  (scipy_corr))
print('******运行结束！************')

In [ ]:
# plot the result correlation OT

import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8),dpi=200)
labelsize = 15
ticksize = 18
dot_size = 80
# x = 'Caudate_R_Frontal_Mid_Orb_R'
# y = 'acc_mean'
sns.regplot(x=reallabel_list, y=predictlabel_list, color="#FC8D62",marker="o", scatter_kws={'s':dot_size}, line_kws={'linewidth':4})
sns.despine()

# plt.xlabel(x, fontsize=labelsize)
# plt.ylabel(y, fontsize=labelsize)

plt.yticks(size=ticksize, fontproperties='Arial')
plt.xticks(size=ticksize, fontproperties='Arial')
leg = plt.legend([])
leg.get_frame().set_linewidth(0.0) 

In [ ]:
# plot the result correlation PL

import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,8),dpi=200)
labelsize = 15
ticksize = 18
dot_size = 80
# x = 'Caudate_R_Frontal_Mid_Orb_R'
# y = 'acc_mean'
sns.regplot(x=reallabel_list_pl, y=predictlabel_list_pl, color="#3E9C7F",marker="o", scatter_kws={'s':dot_size}, line_kws={'linewidth':4})
sns.despine()

# plt.xlabel(x, fontsize=labelsize)
# plt.ylabel(y, fontsize=labelsize)

plt.yticks(size=ticksize, fontproperties='Arial')
plt.xticks(size=ticksize, fontproperties='Arial')
leg = plt.legend([])
leg.get_frame().set_linewidth(0.0) 

In [ ]:
# old permutation test for OT
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso, ElasticNet

scaler = StandardScaler()

# import data
dataset = scaler.fit_transform(selected_feature['OT']['feature']) 
# dataset = selected_feature['OT']['feature']    

y = np.array(behavior_df[behavior_data])[0:30]
############################SVM+LOOVC################
loo = LeaveOneOut()
loo.get_n_splits(dataset)

predictlabel_list = []
reallabel_list = []

# C：C-SVC的惩罚参数C
# kernel: linear, poly, rbf, sigmoid, precomputed
C = 1
clf = SVR(C=C, kernel='linear', gamma='auto', degree=6, epsilon = 0.02)
# clf = lasso
# 用留一法进行验证
for train_index, test_index in loo.split(dataset):
    # 分割训练集
    X_train, X_test = dataset[train_index], dataset[test_index]
    # 分割测试集
    Y_train, Y_test = y[np.array(train_index)], y[np.array(test_index)]
    # 拟合
    clf.fit(X_train, Y_train)
    # 储存训练结果
    predictlabel_list.append(list(clf.predict(X_test)))
    reallabel_list.append(list(Y_test))
reallabel_list = np.squeeze(reallabel_list)
predictlabel_list = np.squeeze(predictlabel_list)
# accurancy = count_right_label / len(label)
scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
acc_0 = scipy_corr[0]
print('******循环结束！************')
print('准确率为：',  (scipy_corr))
print('******运行结束！************')

############################Permutation################
n_permutation = 10000

all_permutation_acc_ot = []
for iter_n in trange(n_permutation):
    # label permutation
    iter_y = y.copy()
    iter_y = np.random.permutation(iter_y)
    ############################SVM+LOOVC################
    loo = LeaveOneOut()
    loo.get_n_splits(dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(dataset):
        # 分割训练集
        X_train, X_test = dataset[train_index], dataset[test_index]
        # 分割测试集
        Y_train, Y_test = iter_y[np.array(train_index)], iter_y[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))

    scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
    all_permutation_acc_ot.append(scipy_corr[0])


In [ ]:
# plot the permutation result for OT

import seaborn as sns
import matplotlib.pyplot as plt
# acc_0 = 0.492
bins_interval= 1/30
bins =np.arange(min(all_permutation_acc_ot) - bins_interval/2, max(all_permutation_acc_ot)+bins_interval/2, bins_interval)
# bins = int((max(all_permutation_acc) - min(all_permutation_acc))/bins_interval)+1
p_value = sum(ms > (acc_0) for ms in all_permutation_acc_ot) / len(all_permutation_acc_ot)
print(p_value)
plt.figure(figsize=(4.5,3),dpi=100)
ax=plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['bottom'].set_color('none')
# sns.kdeplot(all_permutation_acc, shade=True, color='salmon')
# plt.hist(all_permutation_acc, color='salmon', bins = bins)
sns.distplot(all_permutation_acc_ot, bins=bins, color='#FCC3AA',hist_kws={'rwidth': 0.7})   # kdeplot
# plt.title('rbf 02 10k C = '+ str(C) + ',P = ' + str(p_value))
# plt.xlabel('acc')
# plt.xlim([-0.6,0.6])
plt.axvline(acc_0, c="#FC8D62", ls = "dashed",linewidth=2.5)

In [ ]:
# old permutation test for PL
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso, ElasticNet

scaler = StandardScaler()

# import data
dataset = scaler.fit_transform(selected_feature['PL']['feature']) 
# dataset = selected_feature['OT']['feature']    

y = np.array(behavior_df[behavior_data])[30:]
############################SVM+LOOVC################
loo = LeaveOneOut()
loo.get_n_splits(dataset)

predictlabel_list = []
reallabel_list = []

# C：C-SVC的惩罚参数C
# kernel: linear, poly, rbf, sigmoid, precomputed
C = 1
clf = SVR(C=C, kernel='linear', gamma='auto', degree=6, epsilon = 0.02 )
# clf = lasso
# 用留一法进行验证
for train_index, test_index in loo.split(dataset):
    # 分割训练集
    X_train, X_test = dataset[train_index], dataset[test_index]
    # 分割测试集
    Y_train, Y_test = y[np.array(train_index)], y[np.array(test_index)]
    # 拟合
    clf.fit(X_train, Y_train)
    # 储存训练结果
    predictlabel_list.append(list(clf.predict(X_test)))
    reallabel_list.append(list(Y_test))
reallabel_list = np.squeeze(reallabel_list)
predictlabel_list = np.squeeze(predictlabel_list)
# accurancy = count_right_label / len(label)
scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
acc_2 = scipy_corr[0]
print('******循环结束！************')
print('准确率为：',  (scipy_corr))
print('******运行结束！************')

############################Permutation################
n_permutation = 10000

all_permutation_acc_pl = []
for iter_n in trange(n_permutation):
    # label permutation
    iter_y = y.copy()
    iter_y = np.random.permutation(iter_y)
    ############################SVM+LOOVC################
    loo = LeaveOneOut()
    loo.get_n_splits(dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(dataset):
        # 分割训练集
        X_train, X_test = dataset[train_index], dataset[test_index]
        # 分割测试集
        Y_train, Y_test = iter_y[np.array(train_index)], iter_y[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))

    scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
    all_permutation_acc_pl.append(scipy_corr[0])


In [ ]:
# plot permutation test for pl
# acc_2 = 0.101
import seaborn as sns
import matplotlib.pyplot as plt
bins_interval= 1/29
bins =np.arange(min(all_permutation_acc_pl) - bins_interval/2, max(all_permutation_acc_pl)+bins_interval/2, bins_interval)
# bins = int((max(all_permutation_acc) - min(all_permutation_acc))/bins_interval)+1
p_value = sum(ms > (acc_2) for ms in all_permutation_acc_pl) / len(all_permutation_acc_pl)
print(p_value)
plt.figure(figsize=(4.5,3),dpi=100)
ax=plt.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['bottom'].set_color('none')
# sns.kdeplot(all_permutation_acc, shade=True, color='salmon')
# plt.hist(all_permutation_acc, color='salmon', bins = bins)
sns.distplot(all_permutation_acc_pl, bins=bins, color='#8ED2BD',hist_kws={'rwidth': 0.7})   # kdeplot
# plt.title('rbf 02 10k C = '+ str(C) + ',P = ' + str(p_value))
# plt.xlabel('acc')
# plt.xlim([-0.6,0.6])
plt.axvline(acc_2, c="#3E9C7F", ls = "dashed",linewidth = 2.5)

In [ ]:
# new permutation from feature selection for PL
# permutate from select features

#进行单次 OT svm
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
from tqdm import trange

# basic information
threshold = 0.01
behavior_data = 'acc_mean'

# 提取输入数据
y_pl = np.array(behavior_df[behavior_data])[30:]
x_pl = all_data['rest'][30:]

# 创建回归器
# C：C-SVC的惩罚参数C  epsilon：与学习率有关
# kernel: linear, poly, rbf, sigmoid, precomputed
clf = SVR(C=1, kernel='linear', gamma='auto', degree=6, epsilon = 0.1)

#######################################Permutation##############################
n_permutation = 10000

# 创建变量存储每次排列的结果
all_permutation_acc_pl = []
for iter_n in trange(n_permutation):
    # label permutation

    iter_y_pl = y_pl.copy()
    iter_y_pl = np.random.permutation(iter_y_pl)

    # create dict to store selected feature
    selected_feature = {}
    selected_feature['PL'] = {}

    ######################################select features #################################
    
    # print(selected_feature['OT']['feature_name'].shape)

    # Select significant sides   PL
    all_significant_num = []
    for iter_fc in range(x_pl.shape[1]):

        scipy_corr = scipy.stats.pearsonr(x_pl[:,iter_fc], iter_y_pl)

        if scipy_corr[1] < threshold:
            all_significant_num.append(iter_fc)
        # 
    try:
        selected_feature['PL']['feature'] = x_pl[:, np.array(all_significant_num)]
        selected_feature['PL']['feature_name'] = all_fc_name[np.array(all_significant_num)]
    except:
        continue
    # print(selected_feature['PL']['feature_name'].shape)


    ###################################### SVM+LOOVC##################################
   
    # PL
    dataset = selected_feature['PL']['feature']
    loo = LeaveOneOut()
    loo.get_n_splits(dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(dataset):
        # 分割训练集
        X_train, X_test = dataset[train_index], dataset[test_index]
        # 分割测试集
        Y_train, Y_test = iter_y_pl[np.array(train_index)], iter_y_pl[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))

    scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
    all_permutation_acc_pl.append(scipy_corr[0])



In [ ]:
# new permutation from feature selection for OT and PL
# permutate from select features

#进行单次 OT svm
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
from tqdm import trange

# basic information
threshold = 0.01
behavior_data = 'acc_mean'

# 提取输入数据
y_ot = np.array(behavior_df[behavior_data])[0:30]
x_ot = all_data['rest'][0:30]

y_pl = np.array(behavior_df[behavior_data])[30:]
x_pl = all_data['rest'][30:]

# 创建回归器
# C：C-SVC的惩罚参数C  epsilon：与学习率有关
# kernel: linear, poly, rbf, sigmoid, precomputed
clf = SVR(C=1, kernel='linear', gamma='auto', degree=6, epsilon = 0.1)

#######################################Permutation##############################
n_permutation = 10000

# 创建变量存储每次排列的结果
all_permutation_acc_ot = []
all_permutation_acc_pl = []
for iter_n in trange(n_permutation):
    # label permutation
    iter_y_ot = y_ot.copy()
    iter_y_ot = np.random.permutation(iter_y_ot)
    iter_y_pl = y_pl.copy()
    iter_y_pl = np.random.permutation(iter_y_pl)

    # create dict to store selected feature
    selected_feature = {}
    selected_feature['OT'] = {}
    selected_feature['PL'] = {}

    ######################################select features #################################
    # Select significant sides   OT
    all_significant_num = []
    for iter_fc in range(x_ot.shape[1]):

        scipy_corr = scipy.stats.pearsonr(x_ot[:,iter_fc], iter_y_ot)

        if scipy_corr[1] < threshold:
            all_significant_num.append(iter_fc)
        # 
    selected_feature['OT']['feature'] = x_ot[:, np.array(all_significant_num)]
    selected_feature['OT']['feature_name'] = all_fc_name[np.array(all_significant_num)]

    # print(selected_feature['OT']['feature_name'].shape)

    # Select significant sides   PL
    all_significant_num = []
    for iter_fc in range(x_pl.shape[1]):

        scipy_corr = scipy.stats.pearsonr(x_pl[:,iter_fc], iter_y_pl)

        if scipy_corr[1] < threshold:
            all_significant_num.append(iter_fc)
        # 
    selected_feature['PL']['feature'] = x_pl[:, np.array(all_significant_num)]
    selected_feature['PL']['feature_name'] = all_fc_name[np.array(all_significant_num)]
    # print(selected_feature['PL']['feature_name'].shape)


    ###################################### SVM+LOOVC##################################
    # OT
    dataset = selected_feature['OT']['feature']
    loo = LeaveOneOut()
    loo.get_n_splits(dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(dataset):
        # 分割训练集
        X_train, X_test = dataset[train_index], dataset[test_index]
        # 分割测试集
        Y_train, Y_test = iter_y_ot[np.array(train_index)], iter_y_ot[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))

    scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
    all_permutation_acc_ot.append(scipy_corr[0])

    # PL
    dataset = selected_feature['PL']['feature']
    loo = LeaveOneOut()
    loo.get_n_splits(dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(dataset):
        # 分割训练集
        X_train, X_test = dataset[train_index], dataset[test_index]
        # 分割测试集
        Y_train, Y_test = iter_y_pl[np.array(train_index)], iter_y_pl[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))

    scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
    all_permutation_acc_pl.append(scipy_corr[0])



In [ ]:
# new permutation from feature selection for OT
# permutate from select features

#进行单次 OT svm
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
from tqdm import trange

# basic information
threshold = 0.01
behavior_data = 'acc_mean'

# 提取输入数据
y_ot = np.array(behavior_df[behavior_data])[0:30]
x_ot = all_data['rest'][0:30]

# 创建回归器
# C：C-SVC的惩罚参数C  epsilon：与学习率有关
# kernel: linear, poly, rbf, sigmoid, precomputed
clf = SVR(C=1, kernel='linear', gamma='auto', degree=6, epsilon = 0.1)

#######################################Permutation##############################
n_permutation = 10000

# 创建变量存储每次排列的结果
all_permutation_acc_ot = []
for iter_n in trange(n_permutation):
    # label permutation
    iter_y_ot = y_ot.copy()
    iter_y_ot = np.random.permutation(iter_y_ot)

    # create dict to store selected feature
    selected_feature = {}
    selected_feature['OT'] = {}

    ######################################select features #################################
    # Select significant sides   OT
    all_significant_num = []
    for iter_fc in range(x_ot.shape[1]):

        scipy_corr = scipy.stats.pearsonr(x_ot[:,iter_fc], iter_y_ot)

        if scipy_corr[1] < threshold:
            all_significant_num.append(iter_fc)
        
        
    try:
        selected_feature['OT']['feature'] = x_ot[:, np.array(all_significant_num)]
        selected_feature['OT']['feature_name'] = all_fc_name[np.array(all_significant_num)]
    except:
        continue

    # print(selected_feature['OT']['feature_name'].shape)

    ###################################### SVM+LOOVC##################################
    # OT
    dataset = selected_feature['OT']['feature']
    loo = LeaveOneOut()
    loo.get_n_splits(dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(dataset):
        # 分割训练集
        X_train, X_test = dataset[train_index], dataset[test_index]
        # 分割测试集
        Y_train, Y_test = iter_y_ot[np.array(train_index)], iter_y_ot[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))

    scipy_corr = scipy.stats.spearmanr(reallabel_list, predictlabel_list)
    all_permutation_acc_ot.append(scipy_corr[0])


In [ ]:
#每个特征都删一次,得到新的相关性
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso, ElasticNet

scaler = StandardScaler()

# import data
dataset = scaler.fit_transform(selected_feature['OT']['feature'])
# dataset = selected_feature['rest']['feature']    
y = np.array(behavior_df[behavior_data])[0:30]

# C：C-SVC的惩罚参数C
# kernel: linear, poly, rbf, sigmoid, precomputed
C = 1
clf = SVR(C=C, kernel='linear', gamma='auto', degree=6, epsilon = 0.02 )


############################iter_feature################

# 生成储存变量
all_acc = []
all_feature_p = []
n_feature = dataset.shape[1]
for iter_n in trange(n_feature):
    # delete one feature 
    iter_dataset = dataset.copy()
    iter_dataset = np.delete(iter_dataset, iter_n, axis=1)

    ############################SVM+LOOVC################ 先进行一次，查看permutation前的正确率
    loo = LeaveOneOut()
    loo.get_n_splits(iter_dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(iter_dataset):
        # 分割训练集
        X_train, X_test = iter_dataset[train_index], iter_dataset[test_index]
        # 分割测试集
        Y_train, Y_test = y[np.array(train_index)], y[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))
    # 生成留一法后的正确率
    scipy_corr = scipy.stats.spearmanr(predictlabel_list, reallabel_list)

    all_acc.append(scipy_corr[0])


In [ ]:
# 根据acc的差距展示重要的FC
max_acc = 0.575
acc_difference = max_acc - np.array(all_acc) 

sorted_index = np.argsort(acc_difference)
sorted_p_difference = acc_difference[sorted_index]
sorted_featurename = selected_feature['OT']['feature_name'][sorted_index]

import matplotlib.pyplot as plt
plt.figure(figsize=(8,4),dpi=300)
plt.barh(range(10), sorted_p_difference[-10:], tick_label=sorted_featurename[-10:], color='#4F83D9', height=0.6)
# plt.xlabel('p-value difference')
plt.xlabel('Feature importance (Acc difference)')

In [ ]:
# all_feature_p_value = np.load(result_path + '/' + 'all_feature_p_value.npy', allow_pickle=True).item()
max_p = 0.492
p_difference = all_feature_p_value['pvalue'] - max_p

sorted_index = np.argsort(p_difference)
sorted_p_difference = p_difference[sorted_index]
sorted_featurename = all_feature_p_value['feature_name'][sorted_index]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,4),dpi=300)
plt.barh(range(10), sorted_p_difference[-10:], tick_label=sorted_featurename[-10:], color='salmon', height=0.6)
# plt.xlabel('p-value difference')
plt.xlabel('feature importance (p-value difference)')

In [ ]:
np.array(all_feature_p_value['acc']).shape

In [ ]:
feature_importace_data_10 = pd.DataFrame()
feature_importace_data_10['value'] = sorted_p_difference[-7:]
feature_importace_data_10['brain_region1'] = ['Paracentral_Lobule_L', 'Temporal_Pole_Sup_L', 
                        'Insula_L', 'Putamen_R', 'Thalamus_R', 'Caudate_R', 'Temporal_Pole_Mid_R']
feature_importace_data_10['brain_region2'] = [ 'Cingulum_Post_R', 'Angular_L', 
                        'Frontal_Mid_L', 'Frontal_Mid_Orb_R', 'Hippocampus_L', 'Frontal_Mid_Orb_R', 'Angular_L']
feature_importace_data_10.to_csv(result_path + '/' + 'feature_importace_data_acc_7.csv')

In [ ]:
sorted_p_difference[-10:], tick_label=sorted_featurename[-10:]

In [ ]:
#保留指定行
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import scipy.io as scio  # 用于读取matlab格式的数据
import copy
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Lasso, ElasticNet

scaler = StandardScaler()

# import data
dataset = scaler.fit_transform(selected_feature['rest']['feature'])
# dataset = selected_feature['rest']['feature']    
############################SVM#######################

# C：C-SVC的惩罚参数C
# kernel: linear, poly, rbf, sigmoid, precomputed
C = 1
clf = SVC(C=C, kernel='rbf', gamma='auto')

############################Permutation################
n_feature = dataset.shape[1]

all_feature_acc = []

all_acc = []
select_feature = 30
for iter_feature in range(select_feature):
    # delete one feature 
    iter_dataset = dataset.copy()
    # iter_dataset = np.delete(iter_dataset, sorted_index[0:1], axis=1)
    iter_dataset = iter_dataset[:,sorted_index[-iter_feature:]]
    ############################SVM+LOOVC################
    loo = LeaveOneOut()
    loo.get_n_splits(iter_dataset)
    predictlabel_list = []
    reallabel_list = []

    # 用留一法进行验证
    for train_index, test_index in loo.split(iter_dataset):
        # 分割训练集
        X_train, X_test = iter_dataset[train_index], iter_dataset[test_index]
        # 分割测试集
        Y_train, Y_test = label[np.array(train_index)], label[np.array(test_index)]
        # 拟合
        clf.fit(X_train, Y_train)
        # 储存训练结果
        predictlabel_list.append(list(clf.predict(X_test)))
        reallabel_list.append(list(Y_test))

    iter_accuracy = accuracy_score(reallabel_list,predictlabel_list)
    # print(iter_accuracy)
    all_acc.append(iter_accuracy)
